# 准备config配置文件-PSPNet语义分割模型

同济子豪兄 2023-2-13 6-11 6-25

## 进入MMSegmentation主目录

In [1]:
import os
os.chdir('mmsegmentation')

## 指定模型config配置文件

In [2]:
!rm -rf configs/pspnet/pspnet_r50-d8_4xb2-40k_ZihaoDataset.py # 删除原有文件

!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20230130-mmseg/watermelon/configs/pspnet_r50-d8_4xb2-40k_ZihaoDataset.py -P configs/pspnet

--2023-06-26 21:19:31--  https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20230130-mmseg/watermelon/configs/pspnet_r50-d8_4xb2-40k_ZihaoDataset.py
正在连接 172.16.0.13:5848... 已连接。
已发出 Proxy 请求，正在等待回应... 200 OK
长度： 344 [binary/octet-stream]
正在保存至: “configs/pspnet/pspnet_r50-d8_4xb2-40k_ZihaoDataset.py”

pspnet_r50-d8_4xb2- 100%[===================>]     344  --.-KB/s    用时 0s      

2023-06-26 21:19:31 (9.46 MB/s) - 已保存 “configs/pspnet/pspnet_r50-d8_4xb2-40k_ZihaoDataset.py” [344/344])



> 也可以选择`configs`目录里的其它模型，但需要将里面的数据集配置文件修改为`../_base_/datasets/ZihaoDataset_pipeline.py`

## 载入config配置文件

In [3]:
from mmengine import Config
cfg = Config.fromfile('./configs/pspnet/pspnet_r50-d8_4xb2-40k_ZihaoDataset.py')

## 修改config配置文件

In [4]:
# 类别个数
NUM_CLASS = 6

In [5]:
cfg.norm_cfg = dict(type='BN', requires_grad=True) # 只使用GPU时，BN取代SyncBN
cfg.crop_size = (512, 512)
cfg.model.data_preprocessor.size = cfg.crop_size
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg

# 模型 decode/auxiliary 输出头，指定为类别个数
cfg.model.decode_head.num_classes = NUM_CLASS
cfg.model.auxiliary_head.num_classes = NUM_CLASS

# 训练 Batch Size
cfg.train_dataloader.batch_size = 4

# cfg.test_dataloader = cfg.val_dataloader

# 结果保存目录
cfg.work_dir = './work_dirs/ZihaoDataset-PSPNet'

cfg.train_cfg.max_iters = 40000 # 训练迭代次数

cfg.train_cfg.val_interval = 500 # 评估模型间隔

cfg.default_hooks.logger.interval = 100 # 日志记录间隔

cfg.default_hooks.checkpoint.interval = 2500 # 模型权重保存间隔

cfg.default_hooks.checkpoint.max_keep_ckpts = 2 # 最多保留几个模型权重

cfg.default_hooks.checkpoint.save_best = 'mIoU' # 保留指标最高的模型权重

# 随机数种子
cfg['randomness'] = dict(seed=0)

## 查看完整config配置文件

In [6]:
# print(cfg.pretty_text)

## 保存最终的config配置文件

In [7]:
cfg.dump('ZihaoDataset_PSPNet_20230626.py')

这个文件也可以直接下载：https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20230130-mmseg/watermelon/configs/ZihaoDataset_PSPNet_20230626.py